# Lab 7

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_07.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

## Datasets

The datasets being used in the lab are listed below:

- [TIGER: US Census Counties](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties)
- [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)
- [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
- [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ)

In [ ]:
import ee
import geemap

ee.Initialize()

## Question 1

Create a fishnet with a 4-degree interval based on the extent of `[-112.5439, 34.0891, -85.0342, 49.6858]`. Use the fishnet to download the Landsat 7 image tiles by the fishnet using the `geemap.download_ee_image_tiles()` function. Relevant Earth Engine assets:

-   `ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')`

![](https://i.imgur.com/L1IH3fq.png)

In [ ]:
m = geemap.Map(center=[40, -100], zoom=4)
image = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003").select(["B4", "B3", "B2"])

vis_params = {"min": 0, "max": 255, "gamma": [0.95, 1.1, 1]}
m.addLayer(image, vis_params, "Landsat")
m

In [ ]:
region = ee.Geometry.BBox(-112.5439, 34.0891, -85.0342, 49.6858)
fc = ee.FeatureCollection(region)
fishnet = geemap.fishnet(fc.geometry(), rows=4, cols=7, delta=0.5)
style = {"color": "ffff00ff", "fillColor": "00000000"}
m.addLayer(fishnet.style(**style), {}, "Fishnet")

In [ ]:
outdir = "tiles"
geemap.download_ee_image_tiles(
    image, fishnet, outdir, prefix="Landsat_", crs="EPSG:4326", scale=120
)

## Question 2

Create annual cloud-free Landsat imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer. 

![](https://i.imgur.com/MN2UXHx.png)

In [ ]:
fc = ee.FeatureCollection("TIGER/2018/Counties")
subset = fc.filter(ee.Filter.eq("NAME", "Knox"))
knox = subset.filter(ee.Filter.eq("STATEFP", "47"))

In [ ]:
m = geemap.Map()
style = {"color": "ff0000ff", "width": 2, "lineType": "solid", "fillColor": "00000000"}
m.addLayer(knox.style(**style), {}, "Knox County, TN")
m.centerObject(knox, 10)
m

In [ ]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate(
    "2021-01-01", "2022-01-01"
)

composite = ee.Algorithms.Landsat.simpleComposite(collection)

vis_params = {"bands": ["B5", "B4", "B3"], "max": 128}

subset = composite.clip(knox)
m.addLayer(subset, vis_params, "Landsat 8")

In [ ]:
text = "Made by Zhanchao Yang"
m.add_text(text, position="bottomright")

In [ ]:
region = m.user_roi  # darw a rectangular of the county you selected
geemap.ee_export_image(
    composite, filename="landsat_composite.tif", region=region, scale=240
)

## Question 3

Create annual cloud-free Sentinel-2 imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process. Narrow down the date range (e.g., summer months) to reduce the number of images, which can avoid memory errors.

![](https://i.imgur.com/r5RQlEJ.png)

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2021-01-01", "2021-01-20")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
    .filterBounds(knox)
    .select(["B4", "B3", "B2"])
)
image = collection.first()
vis_params = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 2000}

Map.addLayer(image, vis_params, "Sentinel-2")
Map.centerObject(knox, zoom=10)
Map

In [ ]:
subset = image.clip(knox)
Map.addLayer(subset, vis_params, "Sentinel-2")

### Detailed example found in notebook

## Question 4

Create annual cloud-free NAIP imagery for the years 2010-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process. 

![](https://i.imgur.com/h66FC8h.png)

### Example can be found in notebook

## Question 5

Download a US county of your choice and save it as a shapefile or GeoJSON file. 

![](https://i.imgur.com/PuK2Vp3.png)

In [ ]:
r = geemap.Map()
r.addLayer(knox.style(**style), {}, "Knox County, TN")
r.centerObject(knox, 10)
r

In [ ]:
geemap.ee_export_vector(knox, filename="knox.geojson")

In [ ]:
text = "Made by Zhanchao Yang"
r.add_text(text, position="bottomright")